In [16]:
import os
import glob
import psycopg2
import pandas as pd
from sqlqueries import *
from datetime import datetime
from createtables import *

In [17]:
conn=psycopg2.connect("host=localhost dbname=sparkifydb user=student password=student")
cur=conn.cursor()

In [18]:
def get_files(filepath):
    all_files=[]
    for root,dirs,files in os.walk(filepath):
        files=glob.glob(os.path.join(root,'*.json'))
        for f in files:
            all_files.append(os.path.abspath(f))
    return all_files
        

In [19]:
song_files=get_files("data/song_data")

df=pd.read_json(song_files[0],lines=True)

artist_data=(df[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']].values).tolist()

cur.execute(artist_table_insert,artist_data[0])
conn.commit()

song_data=(df[['song_id','title','artist_id','year','duration']].values).tolist()

cur.execute(song_table_insert,song_data[0])
conn.commit()




In [5]:
song_data[0]

['SOINLJW12A8C13314C', 'City Slickers', 'AR8IEZO1187B99055E', 2008, 149.86404]

In [20]:
log_files=get_files("data/log_data")

df1=pd.read_json(log_files[0],lines=True)

df=df1[df1['page']=='NextSong']

t=df['ts'].apply(lambda x: datetime.fromtimestamp(x/1000))

column_labels=['time','hour','day','week','month','year','weekday']

df2=pd.DataFrame(columns=column_labels)

df2['time']=t
df2['hour']=t.dt.hour
df2['day']=t.dt.day
df2['week']=t.dt.week
df2['month']=t.dt.month
df2['year']=t.dt.year
df2['weekday']=t.dt.weekday

time_df=df2


for i,row in time_df.iterrows():
    cur.execute(time_table_insert,list(row))
    conn.commit()


user_df1=df[['userId','firstName','lastName','gender','level']]

user_df=user_df1[user_df1['userId']!='']

for i,row in user_df.iterrows():
    cur.execute(user_table_insert,list(row))
    conn.commit()

In [21]:
for i,row in df.iterrows():
    cur.execute(song_select,(row.song,row.artist,row.length))
    results=cur.fetchone()
    
    #print(results)
    if results:
        song_id,artist_id=results
    else:
        song_id,artist_id=None,None
    
    songplay_data=(pd.to_datetime(row.ts,unit='ms'),int(row.userId),row.level,song_id,artist_id,row.sessionId,row.location,row.userAgent)
    #songplay_data=(row.level)
    cur.execute(songplay_table_insert,songplay_data)   
    conn.commit()
    
    
    
    

In [15]:
conn.close()